In [111]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
import keras
from keras.models import load_model
from keras import backend as K
from keras import models
from keras.layers import Dense
import tensorflow as tf
from tensorflow.keras.utils import to_categorical

from keras_tuner.tuners import RandomSearch

In [112]:
from dataset import load_dataset, load_labels, convert_to_epochs
from features import time_series_features, nonlinear_features, entropy_features, hjorth_features, freq_band_features

In [113]:
num_classes = 2
channels = 32
sfreq = 128

In [114]:
dataset_ = load_dataset(raw = False)
dataset = convert_to_epochs(dataset_, channels, sfreq)

In [115]:
features = time_series_features(dataset, channels)
# freq_bands = np.array([1, 4, 8, 13, 31, 50])
# features = freq_band_features(dataset, channels, sfreq, freq_bands)
# features = nonlinear_features(dataset, channels)
# features = hjorth_features(dataset, channels, sfreq)
# features = entropy_features(dataset, channels, sfreq)
data = features

In [116]:
labels = load_labels()
label = pd.concat([labels['t1_math'], labels['t2_math'],
                  labels['t3_math']]).to_numpy()
label = label.repeat(dataset.shape[1])

# Linear Regression

In [117]:
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
lr_clf = LogisticRegression(max_iter=1000).fit(x_train, y_train)
y_pred = lr_clf.predict(x_test)
y_true = y_test


In [118]:
scores_lr = lr_clf.score(x_test, y_test)
precision_lr = metrics.precision_score(y_true, y_pred, average='macro')
recall_lr = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_lr = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_lr)
print('precision is:', precision_lr)
print('recall is:', recall_lr)
print('f1_score is:', f1_score_lr)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.5494949494949495
precision is: 0.5476043130970776
recall is: 0.5494949494949495
f1_score is: 0.5494949494949495


array([[304, 223],
       [223, 240]])

# KNN Classifier

In [119]:
K.clear_session()
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
scaler = StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train)
x_test = scaler.transform(x_test)
neigh_clf = KNeighborsClassifier(n_neighbors=2)
neigh_clf.fit(x_train, y_train)
y_pred = neigh_clf.predict(x_test)
y_true = y_test


In [120]:
scores_neigh = neigh_clf.score(x_test, y_test)
precision_neigh = metrics.precision_score(y_true, y_pred, average='macro')
recall_neigh = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_neigh = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_neigh)
print('precision is:', precision_neigh)
print('recall is:', recall_neigh)
print('f1_score is:', f1_score_neigh)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.5414141414141415
precision is: 0.533665024470351
recall is: 0.5414141414141415
f1_score is: 0.5111951489516796


array([[407, 120],
       [334, 129]])

# SVM Classifier

In [121]:
x_train, x_test, y_train, y_test = train_test_split(
    data, label, test_size=0.33, random_state=42)
svm_clf = SVC(C=10, kernel='rbf')
svm_clf.fit(x_train, y_train)
y_pred = svm_clf.predict(x_test)
y_pred_train = svm_clf.predict(x_train)
y_true = y_test


In [122]:
scores_svm_train = svm_clf.score(x_train, y_train)
precision_svm_train = metrics.precision_score(y_train, y_pred_train, average='macro')
recall_svm_train = metrics.recall_score(y_train, y_pred_train, average='micro')
f1_score_svm_train = metrics.f1_score(y_train, y_pred_train, average='weighted')
print('accuracy is:', scores_svm_train)
print('precision is:', precision_svm_train)
print('recall is:', precision_svm_train)
print('f1_score is:', f1_score_svm_train)
# metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.5865671641791045
precision is: 0.5866982757435413
recall is: 0.5866982757435413
f1_score is: 0.5800111646532097


In [123]:
scores_svm = svm_clf.score(x_test, y_test)
precision_svm = metrics.precision_score(y_true, y_pred, average='macro')
recall_svm = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_svm = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_svm)
print('precision is:', precision_svm)
print('recall is:', recall_svm)
print('f1_score is:', f1_score_svm)
metrics.confusion_matrix(y_true, y_pred)


accuracy is: 0.498989898989899
precision is: 0.4924366754449575
recall is: 0.498989898989899
f1_score is: 0.4947310979038168


array([[311, 216],
       [280, 183]])

# Neural Network

In [124]:
K.clear_session()
y_v = label
y_v = to_categorical(y_v, num_classes)
x_train, x_test, y_train, y_test = train_test_split(
    data, y_v, test_size=0.33, random_state=42)

In [125]:
# def model_builder(hp):
#     model = keras.Sequential()
#     model.add(keras.Input(shape=(x_train.shape[1],)))

#     for i in range(hp.Int('layers', 2, 6)):
#        s
#                                     activation=hp.Choice('act_' + str(i), ['relu', 'sigmoid'])))

#     model.add(keras.layers.Dense(num_classes, activation='softmax', name='out'))
    
#     hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])
    
#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
#                 loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#                 metrics=['accuracy'])
#     return model

In [126]:
# tuner = RandomSearch(
#     model_builder,
#     objective = 'val_accuracy',
#     max_trials = 4,
#     executions_per_trial = 3
# )

In [127]:
x_in = keras.Input(shape=(x_train.shape[1],))
x = Dense(1000, kernel_initializer='normal', activation='relu')(x_in)
x = Dense(300, kernel_initializer='normal', activation='relu')(x)
x = Dense(200, kernel_initializer='normal', activation='relu')(x)
x = Dense(50, kernel_initializer='normal', activation='relu')(x)
out_layer = Dense(num_classes, activation='softmax', name='out')(x)
model = keras.Model(x_in, out_layer)
#         model.summary()
# Compile model
#         loss_fn = keras.losses.SparseCategoricalCrossentropy()
#         loss_fn = keras.losses.categorical_crossentropy()

In [128]:
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=tf.keras.optimizers.Adam(),
              metrics=['accuracy'])
model.fit(x_train, y_train, epochs=100, batch_size=128, verbose=0)
scores_dnn = model.evaluate(x_test, y_test, verbose=0)

2022-10-28 13:39:16.410623: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.
2022-10-28 13:39:33.314135: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [129]:
y_pred = model.predict(x_test)
y_true = y_test
y_pred = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_true, axis=1)

2022-10-28 13:39:33.653421: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


In [130]:
precision_dnn = metrics.precision_score(y_true, y_pred, average='macro')
recall_dnn = metrics.recall_score(y_true, y_pred, average='micro')
f1_score_dnn = metrics.f1_score(y_true, y_pred, average='weighted')
print('accuracy is:', scores_dnn[1])
print('precision is:', precision_dnn)
print('recall is:', recall_dnn)
print('f1_score is:', f1_score_dnn)
metrics.confusion_matrix(y_true, y_pred)

accuracy is: 0.5323232412338257
precision is: 0.26616161616161615
recall is: 0.5323232323232323
f1_score is: 0.36985411131752594


/opt/homebrew/Caskroom/miniforge/base/envs/init/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


array([[527,   0],
       [463,   0]])